# Fractional Diving example

Fractional Diving is a diving heuristic, that bounds the variable $x_j$ with lowest fractionality $f(x_j)$ to the nearest integer.

## Installing the packages

In the following we use the `GurobiSolver` and thus need to install gurobi.

In [ ]:
%%capture
!pip install gurobipy

Afterwards, we can install hips.

In [ ]:
%%capture
!pip install https://github.com/cxlvinchau/hips/archive/master.zip

## Example

First add the required imports:

In [ ]:
from hips import HIPSArray, VarTypes
from hips.models import MIPModel
from hips.solver import GurobiSolver
from hips.heuristics import FractionalDiving 

Next we initialize a helper function, which allows us to load a model. The model represents the following MIP:
$$
\begin{array}{lr@{}c@{}r@{}l}
    \text{maximize }   & x_1 + 2 x_2  \\
    \text{subject to } & -3 x_1 + 2 x_2 \leq 2 \\
                       & 2 x_1 + 2 x_2 \leq 7 \\
                       & x_1, x_2 \geq 0 \\
                       & x_1, x_2 \in \mathbb{Z}
\end{array}
$$

In [ ]:
def build_model(mip_model):
    x = mip_model.add_variable("x", VarTypes.INTEGER, lb=0, ub=float("inf"), dim=2)
    constr1 = HIPSArray([-3,2])*x <= 2
    constr2 = HIPSArray([2,2])*x <= 7
    mip_model.add_constraint(constr1)
    mip_model.add_constraint(constr2)
    obj_func = HIPSArray([1,2])*x
    mip_model.set_objective(obj_func)

Now we can load this model:

In [ ]:
mip_model = MIPModel(GurobiSolver())
build_model(mip_model)

Next we create the `FractionalDiving` and compute a solution. For the first run we will override the `trivially_rounding` method, so we get the full effect of the heuristic.

In [ ]:
%matplotlib inline
heur = FractionalDiving(mip_model)
heur._round_trivially = lambda :False
heur.compute()

print("Status: {}".format(heur.get_status()))
print("Found solution: {}".format(heur.get_objective_value()))
print("With Variable values: {}".format({var: heur.variable_solution(var) for var in mip_model.get_variables()}))
heur.tracker.plot("objective value")

Now we will use the `FractionalDiving` with the full functionality of `trivially_rounding`. Trivially rounding decides if a variable can be rounded up or down without leaving the feasible region. In the example, we can therefore find the solution after the first iteration.

In [ ]:
mip_model = MIPModel(GurobiSolver())
build_model(mip_model)

In [ ]:
%matplotlib inline
heur = FractionalDiving(mip_model)
heur.compute()

print("Status: {}".format(heur.get_status()))
print("Found solution: {}".format(heur.get_objective_value()))
print("With Variable values: {}".format({var: heur.variable_solution(var) for var in mip_model.get_variables()}))
heur.tracker.plot("objective value")